In [2]:
fn main() {
    let x = 5;  // bind the value 5 to x
    let y = x;  // then make a copy of the value in x and bind it to y
    // We now have two variables, x and y, and both equal 5
    // This is indeed what is happening, because integers are simple values with a known, fixed size, and these two 5 values are pushed onto the stack.
}

In [3]:
fn main() {
    let s1 = String::from("hello");
    let s2 = s1;
}

A String is made up of three parts, shown on the left: a pointer to the memory that holds the contents of the string, a length, and a capacity. This group of data is stored on the stack. On the right is the memory on the heap that holds the contents.

<img src="imgs/img0.svg" width="40%" height="40%">
Figure 0: Representation in memory of a String holding the value "hello" bound to s1
</br>
</br>

When we assign s1 to s2, the String data is copied, meaning we copy the pointer, the length, and the capacity that are on the stack. We do not copy the data on the heap that the pointer refers to. In other words, the data representation in memory looks like Figure 1.

<img src="imgs/img1.svg" width="40%" height="40%">
Figure 1: Representation in memory of the variable s2 that has a copy of the pointer, length, and capacity of s1
</br>
</br>

The representation does not look like Figure 2, which is what memory would look like if Rust instead copied the heap data as well. If Rust did this, the operation s2 = s1 could be very expensive in terms of runtime performance if the data on the heap were large.

<img src="imgs/img2.svg" width="40%" height="40%">
Figure 2: Another possibility for what s2 = s1 might do if Rust copied the heap data as well
</br>
</br>

Earlier, we said that when a variable goes out of scope, Rust automatically calls the drop function and cleans up the heap memory for that variable. But Figure 2 shows both data pointers pointing to the same location. This is a problem: when s2 and s1 go out of scope, they will both try to free the same memory. This is known as a double free error and is one of the memory safety bugs we mentioned previously. Freeing memory twice can lead to memory corruption, which can potentially lead to security vulnerabilities.

To ensure memory safety, after the line let s2 = s1;, Rust considers s1 as no longer valid. Therefore, Rust doesn’t need to free anything when s1 goes out of scope. Check out what happens when you try to use s1 after s2 is created; it won’t work:

In [4]:
// To ensure memory safety, after the line let s2 = s1;
// Rust considers s1 as no longer valid. Therefore, Rust doesn’t need to free anything when s1 goes out of scope. 
// Check out what happens when you try to use s1 after s2 is created; it won’t work:
fn main() {
    let s1 = String::from("hello");
    let s2 = s1;

    println!("{}, world!", s1);
}

main()

Error: unused variable: `s2`

Error: borrow of moved value: `s1`

If you’ve heard the terms shallow copy and deep copy while working with other languages, the concept of copying the pointer, length, and capacity without copying the data probably sounds like making a shallow copy. But because Rust also invalidates the first variable, instead of being called a shallow copy, it’s known as a move. In this example, we would say that s1 was moved into s2. So, what actually happens is shown in Figure 3.

<img src="imgs/img3.svg" width="40%" height="40%">
Figure 3: Representation in memory after s1 has been invalidated
</br>
</br>

That solves our problem! With only s2 valid, when it goes out of scope it alone will free the memory, and we’re done.

In addition, there’s a design choice that’s implied by this: Rust will never automatically create “deep” copies of your data. Therefore, any automatic copying can be assumed to be inexpensive in terms of runtime performance.

In [5]:
// If we do want to deeply copy the heap data of the String, not just the stack data, we can use a common method called clone.
fn main() {
    let s1 = String::from("hello");
    let s2 = s1.clone();

    println!("s1 = {}, s2 = {}", s1, s2);
}

// This works just fine and explicitly produces the behavior shown in Figure 2, where the heap data does get copied.
// When you see a call to clone, you know that some arbitrary code is being executed and that code may be expensive. 
// It’s a visual indicator that something different is going on.

main()

s1 = hello, s2 = hello


()

But this code seems to contradict what we just learned: we don’t have a call to clone, but x is still valid and wasn’t moved into y.

The reason is that types such as integers that have a known size at compile time are stored entirely on the stack, so copies of the actual values are quick to make. That means there’s no reason we would want to prevent x from being valid after we create the variable y. In other words, there’s no difference between deep and shallow copying here, so calling clone wouldn’t do anything different from the usual shallow copying, and we can leave it out.

Rust has a special annotation called the Copy trait that we can place on types that are stored on the stackk, 
If a type implements the Copy trait, variables that use it do not move, but rather are trivially copied, making them still valid after assignment to another variable.

Rust won’t let us annotate a type with Copy if the type, or any of its parts, has implemented the Drop trait. If the type needs something special to happen when the value goes out of scope and we add the Copy annotation to that type, we’ll get a compile-time error.

Here are some of the types that implement Copy:
* All the integer types, such as u32.
* The Boolean type, bool, with values true and false.
* All the floating-point types, such as f64.
* The character type, char.
* Tuples, if they only contain types that also implement Copy. For example, (i32, i32) implements Copy, but (i32, String) does not.

In [6]:
fn main() {
    let x = 5;
    let y = x;

    println!("x = {}, y = {}", x, y);
}

main()

x = 5, y = 5


()